In [95]:
from datetime import datetime 
import nltk
import numpy as np
import os, sys
import pandas as pd
import re

In [96]:
folder = os.path.join(os.getcwd(), 'data')
df = pd.DataFrame()
for i in range(1,2):
    file = os.path.join(folder, 'IRAhandle_tweets_'+str(i)+'.csv')
    dfi = pd.read_csv(file)
    df = df.append(dfi)

    
#df = pd.read_csv('IRAhandle_tweets_1.csv')

In [97]:
df = df[df['language'] == 'English']
df['content'].dropna(inplace = True)
df.drop(['tco1_step1', 'tco2_step1', 'tco3_step1', 'alt_external_id', 'article_url'], axis = 1, inplace = True)
df.sort_values(by='publish_date', inplace = True)
df[df['external_author_id'] == 906000000000000000]


,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,retweet,account_category,new_june_2018,tweet_id
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on trial for corruption and you've barely heard a peep fr...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,RightTroll,0,914580356430536707
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump shirt. Judging by his sagging pants the shirt shoul...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,RightTroll,0,914621840496189440
2,906000000000000000,10_GOP,"Daughter of fallen Navy Sailor delivers powerful monologue on anthem protests, burns her NFL pac...",Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,1,RightTroll,0,914623490375979008
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents Cup golf tournament trophy to the people of Florid...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,RightTroll,0,914639143690555392
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandForOurAnthem🇺🇸 https://t.co/czutyGaMQV",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,1,RightTroll,0,914312219952861184
5,906000000000000000,10_GOP,"Dan Bongino: ""Nobody trolls liberals better than Donald Trump."" Exactly! https://t.co/AigV93aC8J",Unknown,English,10/1/2017 2:47,10/1/2017 2:47,1050,9644,247,NaN,Right,0,RightTroll,0,914320835325853696
6,906000000000000000,10_GOP,🐝🐝🐝 https://t.co/MorL3AQW0z,Unknown,English,10/1/2017 2:48,10/1/2017 2:48,1050,9644,248,RETWEET,Right,1,RightTroll,0,914321156466933760
7,906000000000000000,10_GOP,'@SenatorMenendez @CarmenYulinCruz Doesn't matter that CNN doesn't report on your crimes. This w...,Unknown,English,10/1/2017 2:52,10/1/2017 2:53,1050,9644,249,NaN,Right,0,RightTroll,0,914322215537119234
8,906000000000000000,10_GOP,"As much as I hate promoting CNN article, here they are admitting EVERYTHING Trump said about PR ...",Unknown,English,10/1/2017 3:47,10/1/2017 3:47,1050,9646,250,NaN,Right,0,RightTroll,0,914335818503933957
9,906000000000000000,10_GOP,After the 'genocide' remark from San Juan Mayor the narrative has changed though. @CNN fixes it'...,Unknown,English,10/1/2017 3:51,10/1/2017 3:51,1050,9646,251,NaN,Right,0,RightTroll,0,914336862730375170


info:

region: Regio waar de tweet vandaan komt (?)

author: Accountname

language: taal van de tweet

retweet: 0 of 1, dus is het een retweet of niet
account_category: RightTroll', 'NonEnglish', 'Fearmonger', 'LeftTroll', 'Unknown', 'HashtagGamer', 'NewsFeed', 'Commercial'

Updates: aantal twitter berichten gepost?


In [ ]:
pd.options.display.max_colwidth = 100



In [51]:
df['post_type'].unique()

array([nan, 'RETWEET', 'QUOTE_TWEET'], dtype=object)

In [33]:
df.account_category.unique()

array(['RightTroll', 'NonEnglish', 'Fearmonger', 'LeftTroll', 'Unknown',
       'HashtagGamer', 'NewsFeed', 'Commercial'], dtype=object)

In [37]:
import nltk # be sure to have stopwords installed for this using nltk.download_shell()
import pandas as pd 
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')

# install Vader and make sure you download the lexicon as well
sid = SentimentIntensityAnalyzer()
# this step will return an error if you have not installed the lexicon
summary = {"positive":0,"neutral":0,"negative":0}
for x in df['content']: 
    ss = sid.polarity_scores(x)
    if ss["compound"] == 0.0: 
        summary["neutral"] +=1
    elif ss["compound"] > 0.0:
        summary["positive"] +=1
    else:
        summary["negative"] +=1
print(result)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/thomas/nltk_data...


KeyboardInterrupt: 

In [78]:
df

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,retweet,account_category,new_june_2018,tweet_id
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on trial for corruption and you've barely heard a peep fr...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,RightTroll,0,914580356430536707
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump shirt. Judging by his sagging pants the shirt shoul...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,RightTroll,0,914621840496189440
2,906000000000000000,10_GOP,"Daughter of fallen Navy Sailor delivers powerful monologue on anthem protests, burns her NFL pac...",Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,1,RightTroll,0,914623490375979008
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents Cup golf tournament trophy to the people of Florid...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,RightTroll,0,914639143690555392
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandForOurAnthem🇺🇸 https://t.co/czutyGaMQV",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,1,RightTroll,0,914312219952861184
5,906000000000000000,10_GOP,"Dan Bongino: ""Nobody trolls liberals better than Donald Trump."" Exactly! https://t.co/AigV93aC8J",Unknown,English,10/1/2017 2:47,10/1/2017 2:47,1050,9644,247,NaN,Right,0,RightTroll,0,914320835325853696
6,906000000000000000,10_GOP,🐝🐝🐝 https://t.co/MorL3AQW0z,Unknown,English,10/1/2017 2:48,10/1/2017 2:48,1050,9644,248,RETWEET,Right,1,RightTroll,0,914321156466933760
7,906000000000000000,10_GOP,'@SenatorMenendez @CarmenYulinCruz Doesn't matter that CNN doesn't report on your crimes. This w...,Unknown,English,10/1/2017 2:52,10/1/2017 2:53,1050,9644,249,NaN,Right,0,RightTroll,0,914322215537119234
8,906000000000000000,10_GOP,"As much as I hate promoting CNN article, here they are admitting EVERYTHING Trump said about PR ...",Unknown,English,10/1/2017 3:47,10/1/2017 3:47,1050,9646,250,NaN,Right,0,RightTroll,0,914335818503933957
9,906000000000000000,10_GOP,After the 'genocide' remark from San Juan Mayor the narrative has changed though. @CNN fixes it'...,Unknown,English,10/1/2017 3:51,10/1/2017 3:51,1050,9646,251,NaN,Right,0,RightTroll,0,914336862730375170
